In [2]:
import geopandas as gpd

gdf_shapefile = gpd.read_file("../data/raw/pop_income")

In [4]:
import pandas as pd

file_address='../data/landing/rental_merged.csv'
df=pd.read_csv(file_address)

      property_index   Cost  Beds  Baths  Cars  \
0                  0  560.0     2      1   1.0   
1                  1  550.0     2      1   1.0   
2                  2  340.0     1      1   1.0   
3                  4  460.0     3      1   NaN   
4                  5  270.0     1      1   1.0   
...              ...    ...   ...    ...   ...   
9103           10012  950.0     4      3   2.0   
9104           10013   75.0     0      1   1.0   
9105           10014  690.0     3      2   2.0   
9106           10016  700.0     3      1   1.0   
9107           10017  650.0     3      1   1.0   

                                           Address            Property Type  \
0       6/121 Mcdonald Street, Mordialloc VIC 3195  Apartment / Unit / Flat   
1         5/3 Carnarvon Street, Doncaster VIC 3108  Apartment / Unit / Flat   
2            4/10 Cole Street, Noble Park VIC 3174  Apartment / Unit / Flat   
3                  8 Perth Avenue, Albion VIC 3020                    House   
4   

In [6]:
from shapely.geometry import Point

# Split the string to extract latitude and longitude
def parse_coordinates(coord):
    lat, lon = coord.strip("()").split(", ")
    return Point(float(lon), float(lat))  # Note: Longitude comes first in Point()

# Apply the function to create a 'geometry' column
df['geometry'] = df['coordinates'].apply(parse_coordinates)

# Convert to a GeoDataFrame
gdf_coordinates = gpd.GeoDataFrame(df, geometry='geometry')

# Set the CRS to WGS84 (epsg:4326), which is the standard for lat/lon
gdf_coordinates.set_crs(epsg=4326, inplace=True)


,property_index,Cost,Beds,Baths,Cars,Address,Property Type,coordinates,latitude,longitude,...,nearest_parks_latitude,nearest_parks_longitude,straight_line_distance_parks,route_distance_parks,nearest_shopping_name,nearest_shopping_latitude,nearest_shopping_longitude,straight_line_distance_shopping,route_distance_shopping,geometry
0,0,560.0,2,1,1.0,"6/121 Mcdonald Street, Mordialloc VIC 3195",Apartment / Unit / Flat,"(-38.0045428, 145.0884301)",-38.004543,145.088430,...,-38.003822,145.092254,0.344506,0.447,PARKDALE PLAZA,-37.997429,145.084534,0.861520,0.967,POINT (145.08843 -38.00454)
1,1,550.0,2,1,1.0,"5/3 Carnarvon Street, Doncaster VIC 3108",Apartment / Unit / Flat,"(-37.7863384, 145.1237982)",-37.786338,145.123798,...,-37.787044,145.124325,0.091093,0.089,WESTFIELD SHOPPINGTOWN DONCASTER,-37.784703,145.126228,0.280469,1.179,POINT (145.1238 -37.78634)
2,2,340.0,1,1,1.0,"4/10 Cole Street, Noble Park VIC 3174",Apartment / Unit / Flat,"(-37.9523213, 145.1736)",-37.952321,145.173600,...,-37.953229,145.171167,0.236009,0.286,NaN,-37.957347,145.169569,0.661181,1.316,POINT (145.1736 -37.95232)
3,4,460.0,3,1,NaN,"8 Perth Avenue, Albion VIC 3020",House,"(-37.7753959, 144.8154461)",-37.775396,144.815446,...,-37.770949,144.815452,0.494472,1.477,SUNSHINE MARKETPLACE SHOPPING CENTRE,-37.781744,144.830832,1.525338,2.061,POINT (144.81545 -37.7754)
4,5,270.0,1,1,1.0,"49 Dickson Street, Sunshine VIC 3020",House,"(-37.786819, 144.83411450359245)",-37.786819,144.834114,...,-37.786386,144.838049,0.349055,0.400,SUNSHINE PLAZA,-37.782340,144.833646,0.499720,0.865,POINT (144.83411 -37.78682)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9103,10012,950.0,4,3,2.0,"2/1106 Burke Road, Balwyn North VIC 3104",Townhouse,"(-37.7845246, 145.063037)",-37.784525,145.063037,...,-37.785173,145.062976,0.072351,NaN,NaN,-37.793012,145.064709,0.955085,NaN,POINT (145.06304 -37.78452)
9104,10013,75.0,0,1,1.0,"Car Park/228 La Trobe St, Melbourne VIC 3000",Carspace,"(-37.8114585, 144.9582685)",-37.811459,144.958269,...,-37.809238,144.957901,0.248994,NaN,QUEEN VICTORIA MARKET,-37.807499,144.957157,0.450959,NaN,POINT (144.95827 -37.81146)
9105,10014,690.0,3,2,2.0,"4/420 Middleborough Road, Blackburn VIC 3130",Townhouse,"(-37.8209424, 145.1381392)",-37.820942,145.138139,...,-37.820521,145.136005,0.193239,NaN,NaN,-37.820333,145.138674,0.082399,NaN,POINT (145.13814 -37.82094)
9106,10016,700.0,3,1,1.0,Balwyn VIC 3103,House,"(-37.8091737, 145.0833678)",-37.809174,145.083368,...,-37.808777,145.087543,0.369442,NaN,NaN,-37.812684,145.083443,0.390418,NaN,POINT (145.08337 -37.80917)


In [8]:
# Check if CRS is compatible with the shapefile
if gdf_shapefile.crs != gdf_coordinates.crs:
    gdf_coordinates = gdf_coordinates.to_crs(gdf_shapefile.crs)

# Perform the spatial join to assign the polygons to each point
joined_gdf = gpd.sjoin(gdf_coordinates, gdf_shapefile, how="left", predicate="within")


      property_index   Cost  Beds  Baths  Cars  \
0                  0  560.0     2      1   1.0   
0                  0  560.0     2      1   1.0   
0                  0  560.0     2      1   1.0   
0                  0  560.0     2      1   1.0   
0                  0  560.0     2      1   1.0   
...              ...    ...   ...    ...   ...   
9107           10017  650.0     3      1   1.0   
9107           10017  650.0     3      1   1.0   
9107           10017  650.0     3      1   1.0   
9107           10017  650.0     3      1   1.0   
9107           10017  650.0     3      1   1.0   

                                         Address            Property Type  \
0     6/121 Mcdonald Street, Mordialloc VIC 3195  Apartment / Unit / Flat   
0     6/121 Mcdonald Street, Mordialloc VIC 3195  Apartment / Unit / Flat   
0     6/121 Mcdonald Street, Mordialloc VIC 3195  Apartment / Unit / Flat   
0     6/121 Mcdonald Street, Mordialloc VIC 3195  Apartment / Unit / Flat   
0     6/121 Mc

In [9]:
joined_gdf.drop(columns='geometry').to_csv("../data/curated/rental_merged.csv", index=False)